# Lab 2: Building a Data Pipeline - SOLUTIONS

**Introduction to Data Science & Engineering - Day 2**

## Setup

In [ ]:
import numpy as np
import pandas as pd
import sqlite3
import json
import os
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)
print("Libraries loaded successfully!")

## Part 1: Extract — Source Data Generation

In [ ]:
# Source 1: Orders data (simulating CSV)
np.random.seed(42)
n_orders = 1500

start_date = datetime(2024, 1, 1)
order_dates = [start_date + timedelta(days=np.random.randint(0, 365)) for _ in range(n_orders)]

orders_data = {
    'order_id': range(1001, 1001 + n_orders),
    'customer_id': np.random.randint(1, 201, n_orders),
    'product_id': np.random.randint(1, 51, n_orders),
    'order_date': order_dates,
    'quantity': np.random.randint(1, 8, n_orders),
    'unit_price': np.round(np.random.uniform(10, 300, n_orders), 2),
    'discount_pct': np.random.choice([0, 5, 10, 15, 20], n_orders, p=[0.5, 0.2, 0.15, 0.1, 0.05]),
    'store_id': np.random.randint(1, 6, n_orders)
}

orders_df = pd.DataFrame(orders_data)

# Inject some quality issues
orders_df.loc[np.random.choice(orders_df.index, 30, replace=False), 'unit_price'] = np.nan
orders_df.loc[np.random.choice(orders_df.index, 10, replace=False), 'quantity'] = -1
orders_df.loc[np.random.choice(orders_df.index, 5, replace=False), 'customer_id'] = 999  # Invalid customer

print(f"Orders extracted: {len(orders_df)} rows")
orders_df.head()

In [ ]:
# Source 2: Customer data (simulating JSON API response)
customers_json = {
    "customers": [
        {
            "customer_id": i,
            "name": f"Customer_{i:03d}",
            "email": f"customer{i}@email.com",
            "segment": np.random.choice(['Premium', 'Standard', 'Basic']),
            "join_date": (datetime(2020, 1, 1) + timedelta(days=np.random.randint(0, 1460))).strftime('%Y-%m-%d'),
            "city": np.random.choice(['Dublin', 'Cork', 'Galway', 'Limerick', 'Waterford']),
            "country": "Ireland"
        }
        for i in range(1, 201)
    ]
}

customers_df = pd.DataFrame(customers_json['customers'])
customers_df['join_date'] = pd.to_datetime(customers_df['join_date'])

print(f"Customers extracted: {len(customers_df)} rows")
customers_df.head()

In [ ]:
# Source 3: Product catalog (simulating dict/API)
categories = ['Electronics', 'Clothing', 'Home & Garden', 'Books', 'Sports']
products = []
for i in range(1, 51):
    cat = categories[(i - 1) % len(categories)]
    products.append({
        'product_id': i,
        'product_name': f'{cat}_Item_{i:03d}',
        'category': cat,
        'brand': np.random.choice(['BrandA', 'BrandB', 'BrandC', 'BrandD']),
        'cost_price': round(np.random.uniform(5, 150), 2),
        'weight_kg': round(np.random.uniform(0.1, 20), 1)
    })

products_df = pd.DataFrame(products)
print(f"Products extracted: {len(products_df)} rows")
products_df.head()

In [ ]:
# Source 4: Store reference data
stores_data = {
    'store_id': [1, 2, 3, 4, 5],
    'store_name': ['Dublin Central', 'Cork Main', 'Galway West', 'Online', 'Limerick Hub'],
    'store_type': ['Physical', 'Physical', 'Physical', 'Online', 'Physical'],
    'region': ['East', 'South', 'West', 'National', 'South']
}
stores_df = pd.DataFrame(stores_data)
print(f"Stores extracted: {len(stores_df)} rows")
stores_df

## Exercise 2.1: Quality Assertions - SOLUTION

In [ ]:
def run_quality_checks(df, checks, source_name):
    """Run a list of quality checks and report results."""
    print(f"\n{'='*50}")
    print(f"Quality Report: {source_name}")
    print(f"{'='*50}")
    
    all_passed = True
    for check_name, check_fn, is_critical in checks:
        try:
            result = check_fn(df)
            status = "PASS" if result else "FAIL"
            marker = "\u2713" if result else "\u2717"
            level = "CRITICAL" if (not result and is_critical) else ""
            print(f"  {marker} {check_name}: {status} {level}")
            if not result and is_critical:
                all_passed = False
        except Exception as e:
            print(f"  \u2717 {check_name}: ERROR - {e}")
            all_passed = False
    
    return all_passed

order_checks = [
    ("Not empty", lambda df: len(df) > 0, True),
    ("No null order_ids", lambda df: df['order_id'].notna().all(), True),
    ("Unique order_ids", lambda df: df['order_id'].is_unique, True),
    ("No null prices", lambda df: df['unit_price'].notna().all(), False),
    ("Positive quantities", lambda df: (df['quantity'] > 0).all(), False),
    ("Valid customer_ids (1-200)", lambda df: df['customer_id'].between(1, 200).all(), False),
    ("Valid dates", lambda df: pd.to_datetime(df['order_date'], errors='coerce').notna().all(), True),
]

orders_valid = run_quality_checks(orders_df, order_checks, "Orders")

In [ ]:
customer_checks = [
    ("Not empty", lambda df: len(df) > 0, True),
    ("Unique customer_ids", lambda df: df['customer_id'].is_unique, True),
    ("No null emails", lambda df: df['email'].notna().all(), True),
    ("Valid segments", lambda df: df['segment'].isin(['Premium', 'Standard', 'Basic']).all(), True),
]

customers_valid = run_quality_checks(customers_df, customer_checks, "Customers")

## Exercise 3.1: Clean Orders - SOLUTION

In [ ]:
print(f"Orders before cleaning: {len(orders_df)}")

# 1. Handle missing prices - fill with product median
orders_clean = orders_df.copy()
median_price = orders_clean['unit_price'].median()
orders_clean['unit_price'].fillna(median_price, inplace=True)

# 2. Fix negative quantities
orders_clean.loc[orders_clean['quantity'] <= 0, 'quantity'] = 1

# 3. Remove invalid customer_ids
orders_clean = orders_clean[orders_clean['customer_id'].between(1, 200)]

print(f"Orders after cleaning: {len(orders_clean)}")
print(f"Removed {len(orders_df) - len(orders_clean)} invalid rows")

## Exercise 3.2: Enrich and Derive Fields - SOLUTION

In [ ]:
orders_clean['order_date'] = pd.to_datetime(orders_clean['order_date'])

# Revenue calculations
orders_clean['gross_amount'] = orders_clean['quantity'] * orders_clean['unit_price']
orders_clean['discount_amount'] = orders_clean['gross_amount'] * orders_clean['discount_pct'] / 100
orders_clean['net_amount'] = orders_clean['gross_amount'] - orders_clean['discount_amount']

# Date dimensions
orders_clean['order_year'] = orders_clean['order_date'].dt.year
orders_clean['order_month'] = orders_clean['order_date'].dt.month
orders_clean['order_quarter'] = orders_clean['order_date'].dt.quarter
orders_clean['order_day_of_week'] = orders_clean['order_date'].dt.dayofweek
orders_clean['is_weekend'] = orders_clean['order_day_of_week'].isin([5, 6]).astype(int)

print("Derived fields created:")
orders_clean[['order_id', 'gross_amount', 'discount_amount', 'net_amount', 'order_quarter', 'is_weekend']].head()

## Exercise 4.1: Create Dimension Tables - SOLUTION

In [ ]:
# dim_customers
dim_customers = customers_df.copy()
dim_customers = dim_customers.rename(columns={'customer_id': 'customer_key'})

dim_customers['tenure_days'] = (pd.Timestamp('2024-12-31') - dim_customers['join_date']).dt.days
dim_customers['tenure_years'] = (dim_customers['tenure_days'] / 365).round(1)

print(f"dim_customers: {dim_customers.shape}")
dim_customers.head()

In [ ]:
# dim_products
dim_products = products_df.copy()
dim_products = dim_products.rename(columns={'product_id': 'product_key'})

dim_products['margin_tier'] = pd.cut(
    dim_products['cost_price'],
    bins=[0, 30, 80, 200],
    labels=['Low Cost', 'Mid Cost', 'High Cost']
)

print(f"dim_products: {dim_products.shape}")
dim_products.head()

In [ ]:
# dim_time
date_range = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')
dim_time = pd.DataFrame({
    'date_key': date_range,
    'year': date_range.year,
    'quarter': date_range.quarter,
    'month': date_range.month,
    'month_name': date_range.strftime('%B'),
    'week': date_range.isocalendar().week.astype(int),
    'day_of_week': date_range.dayofweek,
    'day_name': date_range.strftime('%A'),
    'is_weekend': date_range.dayofweek.isin([5, 6]).astype(int),
})

print(f"dim_time: {dim_time.shape}")
dim_time.head()

In [ ]:
# dim_stores (already clean)
dim_stores = stores_df.copy()
dim_stores = dim_stores.rename(columns={'store_id': 'store_key'})

print(f"dim_stores: {dim_stores.shape}")
dim_stores

## Exercise 4.2: Create Fact Table - SOLUTION

In [ ]:
fact_orders = orders_clean[['order_id', 'customer_id', 'product_id', 'order_date', 'store_id',
                             'quantity', 'unit_price', 'discount_pct',
                             'gross_amount', 'discount_amount', 'net_amount']].copy()

fact_orders = fact_orders.rename(columns={
    'customer_id': 'customer_key',
    'product_id': 'product_key',
    'order_date': 'date_key',
    'store_id': 'store_key'
})

print(f"fact_orders: {fact_orders.shape}")
print(f"Total Revenue: ${fact_orders['net_amount'].sum():,.2f}")
fact_orders.head()

In [ ]:
# Validate referential integrity
print("Star Schema Validation:")
print(f"  Fact rows: {len(fact_orders)}")
print(f"  Unique customers in fact: {fact_orders['customer_key'].nunique()}")
print(f"  Unique products in fact: {fact_orders['product_key'].nunique()}")

orphan_customers = set(fact_orders['customer_key']) - set(dim_customers['customer_key'])
orphan_products = set(fact_orders['product_key']) - set(dim_products['product_key'])
orphan_stores = set(fact_orders['store_key']) - set(dim_stores['store_key'])

print(f"  Orphan customer keys: {len(orphan_customers)}")
print(f"  Orphan product keys: {len(orphan_products)}")
print(f"  Orphan store keys: {len(orphan_stores)}")

## Exercise 5.1: Load into SQLite - SOLUTION

In [ ]:
db_path = 'ecommerce_warehouse.db'

if os.path.exists(db_path):
    os.remove(db_path)

conn = sqlite3.connect(db_path)

# Load tables
fact_orders_sql = fact_orders.copy()
fact_orders_sql['date_key'] = fact_orders_sql['date_key'].astype(str)

fact_orders_sql.to_sql('fact_orders', conn, index=False, if_exists='replace')
dim_customers.to_sql('dim_customers', conn, index=False, if_exists='replace')
dim_products.to_sql('dim_products', conn, index=False, if_exists='replace')
dim_time_sql = dim_time.copy()
dim_time_sql['date_key'] = dim_time_sql['date_key'].astype(str)
dim_time_sql.to_sql('dim_time', conn, index=False, if_exists='replace')
dim_stores.to_sql('dim_stores', conn, index=False, if_exists='replace')

# Verify
for table in ['fact_orders', 'dim_customers', 'dim_products', 'dim_time', 'dim_stores']:
    count = pd.read_sql(f"SELECT COUNT(*) as cnt FROM {table}", conn).iloc[0, 0]
    print(f"  {table}: {count} rows")

print("\nSQLite database created successfully!")

## Exercise 5.2: Load into Parquet - SOLUTION

In [ ]:
parquet_dir = 'warehouse_parquet'
os.makedirs(parquet_dir, exist_ok=True)

fact_orders.to_parquet(f'{parquet_dir}/fact_orders.parquet', index=False)
dim_customers.to_parquet(f'{parquet_dir}/dim_customers.parquet', index=False)
dim_products.to_parquet(f'{parquet_dir}/dim_products.parquet', index=False)
dim_time.to_parquet(f'{parquet_dir}/dim_time.parquet', index=False)
dim_stores.to_parquet(f'{parquet_dir}/dim_stores.parquet', index=False)

print("Parquet files created:")
for f in os.listdir(parquet_dir):
    size = os.path.getsize(os.path.join(parquet_dir, f))
    print(f"  {f}: {size/1024:.1f} KB")

## Exercise 6.1: Revenue Analytics - SOLUTION

In [ ]:
# Monthly revenue trend
query = """
SELECT 
    t.month_name,
    t.month,
    COUNT(f.order_id) AS total_orders,
    ROUND(SUM(f.net_amount), 2) AS total_revenue,
    ROUND(AVG(f.net_amount), 2) AS avg_order_value
FROM fact_orders f
JOIN dim_time t ON DATE(f.date_key) = DATE(t.date_key)
GROUP BY t.month, t.month_name
ORDER BY t.month
"""
monthly_revenue = pd.read_sql(query, conn)
print(monthly_revenue.to_string(index=False))

In [ ]:
# Top customers by revenue
query = """
SELECT 
    c.name,
    c.segment,
    c.city,
    COUNT(f.order_id) AS orders,
    ROUND(SUM(f.net_amount), 2) AS total_spent,
    ROUND(AVG(f.net_amount), 2) AS avg_order
FROM fact_orders f
JOIN dim_customers c ON f.customer_key = c.customer_key
GROUP BY c.customer_key
ORDER BY total_spent DESC
LIMIT 10
"""
top_customers = pd.read_sql(query, conn)
print(top_customers.to_string(index=False))

In [ ]:
# Category performance by store
query = """
SELECT 
    s.store_name,
    p.category,
    COUNT(f.order_id) AS orders,
    ROUND(SUM(f.net_amount), 2) AS revenue,
    ROUND(AVG(f.discount_pct), 1) AS avg_discount
FROM fact_orders f
JOIN dim_stores s ON f.store_key = s.store_key
JOIN dim_products p ON f.product_key = p.product_key
GROUP BY s.store_name, p.category
ORDER BY s.store_name, revenue DESC
"""
store_category = pd.read_sql(query, conn)
print(store_category.head(15).to_string(index=False))

### Cleanup

In [ ]:
# Close connection and clean up
conn.close()

# Clean up temporary files
import shutil
if os.path.exists(db_path):
    os.remove(db_path)
if os.path.exists(parquet_dir):
    shutil.rmtree(parquet_dir)

print("Cleanup complete!")

## Summary

In this lab, you learned how to:

1. **Extract** data from multiple source formats (CSV, JSON, dict)
2. **Validate** data quality with assertion-based checks
3. **Transform** data — cleaning, enrichment, derived fields
4. **Model** data into a star schema (fact + dimension tables)
5. **Load** into both SQLite (for SQL queries) and Parquet (for analytics)
6. **Analyze** with SQL joins across the star schema

---

*Introduction to Data Science & Engineering | AI Elevate*